In [1]:
import os
from tqdm import tqdm

KAGGLE_ENV = not os.path.exists("/kaggle/.vscode")
ENV_NAME = "kaggle" if KAGGLE_ENV else "local"
print(f"{KAGGLE_ENV=}")
print(f"{ENV_NAME=}")

if KAGGLE_ENV:
    !pip install -U -q plyvel --no-index --find-links=file:///kaggle/input/uspto-gen-wheel/plyvel

    move_dirs = (
        [
        "/kaggle/input/preprocess-all-token-single",
        "/kaggle/input/uspto-rare-tokens-dataset",
        ] 
        + [f"/kaggle/input/complete-db-{i}" for i in range(15)]
        + [f"/kaggle/input/complete-db-v2-{i}" for i in range(5)]
        + [f"/kaggle/input/uspto-ratio-db-{i}" for i in range(20)]
    )

    !mkdir /kaggle/tmp
    for move_dir in tqdm(move_dirs):
        !cp -r {move_dir} /kaggle/tmp/{move_dir.split("/")[-1]}
    !ls /kaggle/tmp

KAGGLE_ENV=False
ENV_NAME='local'


In [2]:
import os
import sys

if KAGGLE_ENV:
    PACKAGE_DIR = "/kaggle/input/uspto-src/src"
else:
    PACKAGE_DIR = "/kaggle/src"
sys.path.append(PACKAGE_DIR)
sys.path.append(os.path.join(PACKAGE_DIR, "Penguin-ML-Library"))

In [19]:
import json
import multiprocessing
import random
import warnings
from collections import Counter
from typing import List, Set, Tuple

import matplotlib.pyplot as plt
import numpy as np
import plyvel
import polars as pl
import yaml
from penguinml.utils.logger import get_logger, init_logger
from penguinml.utils.set_seed import seed_base
from penguinml.utils.timer import Timer
from tqdm import tqdm

import whoosh_utils
from const import INF, KEY2QUERY, NUM_CPU, QUERY2KEY
from db import CompleteDB, SingleTokenDB, TokinezedDB
from solver import HitBlock, SimulatedAnnealing, State
from utils import compute_ap, evaluate

warnings.filterwarnings("ignore")
MODEL_NAME = "baseline"
CFG = yaml.safe_load(open(os.path.join(PACKAGE_DIR, "config.yaml"), "r"))
print(CFG[MODEL_NAME]["execution"]["exp_id"])
CFG["output_dir"] = f"/kaggle/output/{CFG[MODEL_NAME]['execution']['exp_id']}"
# !rm -r {CFG["output_dir"]}
os.makedirs(CFG["output_dir"], exist_ok=True)

init_logger("log.log")
logger = get_logger("main")
seed_base(CFG[MODEL_NAME]["execution"]["seed"])

set seed: 46


exp_0001


In [6]:
DANGER_TYPE = "all"
# DANGER_TYPE = "2hop"
assert DANGER_TYPE in ["all", "2hop"]

if KAGGLE_ENV:
    TRAIN_PATH = "/kaggle/input/uspto-train-index-2500/train2500_seed0.parquet"
    #     TRAIN_PATH = "/kaggle/input/uspto-explainable-ai/test.csv"
    TRAIN_INDEX_PATH = "/kaggle/input/uspto-train-index-2500/index_2500_200k"
    NN_DF_PATH = "/kaggle/input/uspto-explainable-ai/nearest_neighbors.csv"

    # database
    PATENT2RARE_TOKENS_PATH = "/kaggle/tmp/uspto-rare-tokens-dataset/db"

    COMPLETE_DB_PATH = [f"/kaggle/tmp/complete-db-{i}/complete-db-{i}/db" for i in range(15)]
    COMPLETE_INDEX_PATH = [f"/kaggle/input/uspto-complete-index-{i}/index.lz4" for i in range(15)]

    COMPLETE_DB_V2_PATH = [f"/kaggle/tmp/complete-db-v2-{i}/db" for i in range(5)]
    COMPLETE_INDEX_V2_PATH = [f"/kaggle/input/complete-db-index-v2-{i}/index.lz4" for i in range(5)]

    CPC_TOKEN2RANGE_SPLIT = 20
    CPC_TOKEN2RANGE_DB_PATHES = [
        f"/kaggle/tmp/uspto-ratio-db-{i}/ratio-db-{i}/db" for i in range(CPC_TOKEN2RANGE_SPLIT)
    ]
    CPC_TOKEN_INDEX_PATHES = [
        f"/kaggle/input/uspto-ratio-db-index-{i}/index.txt" for i in range(CPC_TOKEN2RANGE_SPLIT)
    ]

    SINGLE_TOKEN_DB_PATH = "/kaggle/tmp/preprocess-all-token-single/db/db"
    SINGLE_TOKEN_INDEX_PATH = "/kaggle/tmp/preprocess-all-token-single/index.lz4"

    TRAIN_MODE = "train" in TRAIN_PATH
    VISUALIZE = False
else:
    TRAIN_PATH = "/kaggle/input/uspto-train-data-2500/train2500_seed0.parquet"
    TRAIN_INDEX_PATH = "/kaggle/input/train-index-2500/index_2500_200k"
    NN_DF_PATH = "/kaggle/input/uspto-boolean-search-optimization/nearest_neighbors.csv"

    # database
    PATENT2RARE_TOKENS_PATH = "/kaggle/input/rare-tokens/db"

    COMPLETE_DB_PATH = [
        f"/kaggle/input/preprocess-complete/split/complete-db-{i}/db" for i in range(15)
    ]
    COMPLETE_INDEX_PATH = [
        f"/kaggle/input/preprocess-complete/split/complete-db-{i}/index.lz4" for i in range(15)
    ]

    COMPLETE_DB_V2_PATH = [
        f"/kaggle/input/preprocess-complete-v2/split/complete-db-{i}/db" for i in range(5)
    ]
    COMPLETE_INDEX_V2_PATH = [
        f"/kaggle/input/preprocess-complete-v2/split/complete-db-{i}/index.lz4" for i in range(5)
    ]

    TOKENIZED_SPLIT = 10
    TOKENIZED_DB_PATHES = [
        f"/kaggle/input/all-index-per-patent/split/tokenized-db-{i}/db"
        for i in range(TOKENIZED_SPLIT)
    ]
    TOKENIZED_INDEX_PATHES = [
        f"/kaggle/input/all-index-per-patent/split/tokenized-db-{i}/index.lz4"
        for i in range(TOKENIZED_SPLIT)
    ]

    CPC_TOKEN2RANGE_SPLIT = 20
    CPC_TOKEN2RANGE_DB_PATHES = [
        f"/kaggle/input/preprocess-cands-ratio/split/ratio-db-{i}/db"
        for i in range(CPC_TOKEN2RANGE_SPLIT)
    ]
    CPC_TOKEN_INDEX_PATHES = [
        f"/kaggle/input/preprocess-cands-ratio/split/ratio-db-{i}/index.txt"
        for i in range(CPC_TOKEN2RANGE_SPLIT)
    ]

    SINGLE_TOKEN_DB_PATH = "/kaggle/input/preprocess-all-token-single/db"
    SINGLE_TOKEN_INDEX_PATH = "/kaggle/input/preprocess-all-token-single/index.lz4"

    TRAIN_MODE = True
    VISUALIZE = False

In [5]:
if TRAIN_PATH.split(".")[-1] == "parquet":
    train = pl.read_parquet(TRAIN_PATH)
else:
    train = pl.read_csv(TRAIN_PATH)

In [9]:
tokenized_db = TokinezedDB(TOKENIZED_DB_PATHES, TOKENIZED_INDEX_PATHES)

In [17]:
with open("/kaggle/input/token-counts/token_counts.json", "r") as f:
    token_counts = json.load(f)

In [35]:
for i, target_ids in tqdm(enumerate(train.iter_rows())):
    target_ids = target_ids[1:]
    token2patents = {}
    for target in target_ids:
        tokens = tokenized_db.get(target)
        for key, this_tokens in tokens.items():
            for token in this_tokens:
                token = f"{KEY2QUERY[key]}:{token}"
                if token not in token2patents:
                    token2patents[token] = set()
                token2patents[token].add(target)

    for token, patents in token2patents.items():
        inner_count = len(patents)
        key, _token = token.split(":")
        all_count = token_counts[QUERY2KEY[key]][_token]
        outer_count = all_count - inner_count

        if outer_count < 500 and inner_count > 10:
            print(token, inner_count, outer_count)
    print("------------")

    if i > 50:
        break

1it [00:00,  4.15it/s]

detd:transpyloric 21 184
detd:lcpufas 24 280
detd:arasco 16 145
detd:dhasco 18 252
detd:enfamil 12 214
detd:lcpufa 25 355
detd:chito 13 375
detd:aribino 15 84
detd:siallyl 13 100
detd:fuco 13 452
detd:gentio 14 466
detd:reconstituteable 11 49
detd:lactoflavin 11 349
detd:ovoflavin 11 81
detd:nicmn 11 73
detd:folacin 14 468
detd:pteroylglutamic 11 349
detd:deoxyadenosylcobalamin 12 232
detd:hydroxycobalamin 11 345
detd:picolonate 12 153
detd:nonabsorbed 12 355
------------
ab:wales 11 357
------------


3it [00:00,  6.23it/s]

------------


4it [00:01,  3.39it/s]

...

------------
------------


49it [00:10,  6.40it/s]

------------
------------


51it [00:10,  4.93it/s]

detd:orthotist 11 201
------------
------------
